In [ ]:
!pip install transformers datasets huggingface-hub[hf-xet]

In [3]:
from transformers import (
    BartForConditionalGeneration,
    BartTokenizer,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
import json

# Load dữ liệu
with open("./vietnamese_news_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Chuyển dữ liệu sang dạng HuggingFace Dataset
train_data = [
    {"text": item["text"], "summary": item["summary"]} for item in data
]
dataset = Dataset.from_list(train_data)

# Load tokenizer và model
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


# Tiền xử lý dữ liệu
def preprocess(example):
    inputs = tokenizer(example["text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(example["summary"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs


tokenized_dataset = dataset.map(preprocess, remove_columns=["text", "summary"])

# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./bart_finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    fp16=True,
    report_to="none",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Huấn luyện
trainer.train()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss
10,3.206700
20,2.429600
30,1.906900
40,1.559700
50,1.700400
60,1.350100
70,1.423000
80,1.362200
90,1.226000
100,1.074400


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=150, training_loss=1.4801757176717123, metrics={'train_runtime': 132.2655, 'train_samples_per_second': 2.268, 'train_steps_per_second': 1.134, 'total_flos': 325065690316800.0, 'train_loss': 1.4801757176717123, 'epoch': 3.0})

In [10]:
tokenizer.save_pretrained("./bart_finetuned")
model.save_pretrained("./bart_finetuned")

In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load lại mô hình và tokenizer đã fine-tune
model_name_or_path = "./bart_finetuned"
tokenizer = BartTokenizer.from_pretrained(model_name_or_path)
model = BartForConditionalGeneration.from_pretrained(model_name_or_path)


def summarize(text, max_length=512, min_length=30):
    inputs = tokenizer(
        [text], max_length=512, truncation=True, return_tensors="pt"
    )
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        num_beams=4,
        length_penalty=2.0,
        no_repeat_ngram_size=3,
        early_stopping=True,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


if __name__ == "__main__":
    text = 'Anh nhập Bệnh viện Đa khoa Xuyên Á TP HCM trong vòng 20 phút từ khi xuất hiện triệu chứng. Các bác sĩ ghi nhận bệnh nhân có dấu hiệu điển hình của đột quỵ, bao gồm chóng mặt, run nửa người bên phải, kèm theo huyết áp cao bất thường ở mức 200/100 mmHg. Kết quả chụp MRI xác định nhồi máu tiểu não bên phải. Ngay lập tức, bệnh nhân được kiểm soát huyết áp và tiêm thuốc tiêu sợi huyết (rTPA) qua đường tĩnh mạch. Loại thuốc này giúp tái thông mạch máu bị tắc, đảm bảo cung cấp máu và oxy kịp thời đến các tế bào não, ngăn chặn tổn thương nghiêm trọng. Ngay sau khi tiêm thuốc, tình trạng chóng mặt của bệnh nhân giảm rõ rệt, triệu chứng run cải thiện, có thể giữ thăng bằng khi ngồi và đi lại. Kết quả chụp CT mạch máu não không ghi nhận hẹp hay tắc mạch máu lớn, cho thấy hiệu quả của việc cấp cứu đúng thời điểm. Hiện, anh có thể tự đi lại và vận động bình thường, xuất viện sau 7 ngày điều trị. Bác sĩ Lê Minh Mẫn, Khoa Nội thần kinh, nhấn mạnh tầm quan trọng của việc nhận biết sớm các dấu hiệu đột quỵ để can thiệp kịp thời. Theo quy tắc F.A.S.T, các dấu hiệu điển hình bao gồm méo miệng, liệt tay, nói khó và cần xác định thời gian để đưa bệnh nhân đến bệnh viện sớm nhất. Tuy nhiên, trong một số trường hợp hiếm gặp như bệnh nhân trên, đột quỵ có thể biểu hiện bằng các triệu chứng không điển hình như chóng mặt, choáng váng, mất thăng bằng, mất thị lực, nhìn mờ hoặc nhìn đôi. Vì vậy, quy tắc nhận biết đột quỵ mở rộng là B.E.F.A.S.T (Balance - mất thăng bằng, Eyes - vấn đề thị lực, Face - méo miệng, Arm - liệt tay, Speech - nói khó, Time - thời gian). Bác sĩ Mẫn khuyến cáo khi phát hiện người thân hoặc người xung quanh có dấu hiệu đột quỵ, cần nhanh chóng đưa họ đến cơ sở y tế có trung tâm đột quỵ gần nhất. Tuyệt đối không áp dụng các phương pháp dân gian, vì điều này có thể làm mất "thời gian vàng", dẫn đến hậu quả nghiêm trọng như hôn mê hoặc thậm chí tử vong. Mỹ Ý'
    print("Tóm tắt:", summarize(text))

Tóm tắt: TP HCM - Chụp MRI xác định nhồi máu tiểu não bên phải, kèm theo huyết áp cao bất thường ở mức 200/100 mmHg.


In [4]:
!zip -r bart_finetuned.zip bart_finetuned

  adding: bart_finetuned/ (stored 0%)
  adding: bart_finetuned/checkpoint-100/ (stored 0%)
  adding: bart_finetuned/checkpoint-100/trainer_state.json (deflated 70%)
  adding: bart_finetuned/checkpoint-100/rng_state.pth (deflated 26%)
  adding: bart_finetuned/checkpoint-100/model.safetensors (deflated 7%)
  adding: bart_finetuned/checkpoint-100/training_args.bin (deflated 53%)
  adding: bart_finetuned/checkpoint-100/generation_config.json (deflated 47%)
  adding: bart_finetuned/checkpoint-100/optimizer.pt (deflated 8%)
  adding: bart_finetuned/checkpoint-100/config.json (deflated 62%)
  adding: bart_finetuned/checkpoint-100/scheduler.pt (deflated 61%)
  adding: bart_finetuned/checkpoint-100/scaler.pt (deflated 64%)
  adding: bart_finetuned/checkpoint-150/ (stored 0%)
  adding: bart_finetuned/checkpoint-150/trainer_state.json (deflated 73%)
  adding: bart_finetuned/checkpoint-150/rng_state.pth (deflated 26%)
  adding: bart_finetuned/checkpoint-150/model.safetensors (deflated 7%)
  adding